<p style="background-color: #ffffcc; padding: 10px; border-radius: 5px;">
The aim of this assignment is to train a simple model to predict the duration of a ride, similar to what we practiced during the lecture.
</p>


<h3>Q1. Data Download</h3>

<p>We'll be using the same <a href="https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page">NYC taxi dataset</a>, but instead of the <strong>Green Taxi Trip Records</strong>, we'll work with the <strong>Yellow Taxi Trip Records</strong>.</p>

<p>Download the data for January and February 2023.</p>

<p>Load the January data. How many columns are present?</p>

<ul>
    <li>16</li>
    <li>17</li>
    <li>18</li>
    <li>19</li>
</ul>

<h3>Q2. Calculating Duration</h3>

<p>Now, let's calculate the <code>duration</code> variable. This variable should represent the duration of a ride in minutes.</p>

<p>What is the standard deviation of the trip durations for January?</p>

<ul>
    <li>32.59</li>
    <li>42.59</li>
    <li>52.59</li>
    <li>62.59</li>
</ul>


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [3]:
#Q1
df_january = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_january.shape[1]

19

In [4]:
#Q2
df_january

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [5]:
df_january['duration'] = (pd.to_datetime(df_january['tpep_dropoff_datetime']) - pd.to_datetime(df_january['tpep_pickup_datetime'])).dt.total_seconds() / 60 # so it minutes now
std_duration = df_january['duration'].std()
round(std_duration, 2)

42.59

<h3>Q3. Removing Outliers</h3>

<p>Next, let's examine the distribution of the <code>duration</code> variable. There are some outliers present, so we'll remove them and keep only the records where the duration is between 1 and 60 minutes (inclusive).</p>

<p>What fraction of records remains after removing the outliers?</p>

<ul>
    <li>90%</li>
    <li>92%</li>
    <li>95%</li>
    <li>98%</li>
</ul>

<h3>Q4. One-Hot Encoding</h3>

<p>Now, let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.</p>

<ul>
    <li>Convert the dataframe to a list of dictionaries (remember to recast the IDs as strings to avoid label encoding).</li>
    <li>Apply a dictionary vectorizer.</li>
    <li>Generate a feature matrix from it.</li>
</ul>

<p>What is the dimensionality of this matrix (i.e., the number of columns)?</p>

<ul>
    <li>2</li>
    <li>155</li>
    <li>345</li>
    <li>515</li>
    <li>715</li>
</ul>


In [6]:
#Q3
df_filtered = df_january[(df_january['duration'] >= 1) &
(df_january['duration'] <= 60)]

In [7]:
fraction_remaining = len(df_filtered) / len(df_january)
round(fraction_remaining * 100, 0)

98.0

In [8]:
#Q4
df_filtered.loc[:, 'PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered.loc[:, 'DOLocationID'] = df_filtered['DOLocationID'].astype(str)

data_dicts = df_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

dv = DictVectorizer(sparse=True)
feature_matrix = dv.fit_transform(data_dicts)

C:\Users\santa\AppData\Local\Temp\ipykernel_21832\3101344167.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_filtered.loc[:, 'PULocationID'] = df_filtered['PULocationID'].astype(str)
C:\Users\santa\AppData\Local\Temp\ipykernel_21832\3101344167.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_filtered.loc[:, 'DOLocationID'] = df_filtered['DOLocationID'].astype(str)


In [9]:
feature_matrix.shape[1]

515

<h3>Q5. Model Training</h3>

<p>Let's now use the feature matrix from the previous step to train a model.</p>

<ul>
    <li>Train a simple linear regression model with default parameters, with <code>duration</code> as the target variable.</li>
    <li>Calculate the RMSE of the model on the training dataset.</li>
</ul>

<p>What is the RMSE on the training data?</p>

<ul>
    <li>3.64</li>
    <li>7.64</li>
    <li>11.64</li>
    <li>16.64</li>
</ul>

<h3>Q6. Model Evaluation</h3>

<p>Next, let's evaluate this model using the validation dataset (February 2023).</p>

<p>What is the RMSE on the validation data?</p>

<ul>
    <li>3.81</li>
    <li>7.81</li>
    <li>11.81</li>
    <li>16.81</li>
</ul>


In [10]:
#Q5
y = df_filtered['duration']

model = LinearRegression()
model.fit(feature_matrix, y)

y_pred = model.predict(feature_matrix)
rmse = np.sqrt(mean_squared_error(y, y_pred))

round(rmse, 2)

7.65

In [11]:
#Q6

In [12]:
df_february = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [13]:
df_february['duration'] = (pd.to_datetime(df_february['tpep_dropoff_datetime']) - pd.to_datetime(df_february['tpep_pickup_datetime'])).dt.total_seconds() / 60

In [14]:
df_february_filtered = df_february[(df_february['duration'] >= 1) & (df_february['duration'] <= 60)]

In [15]:
df_february_filtered['PULocationID'] = df_february_filtered['PULocationID'].astype(str)
df_february_filtered['DOLocationID'] = df_february_filtered['DOLocationID'].astype(str)

C:\Users\santa\AppData\Local\Temp\ipykernel_21832\3579443959.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_february_filtered['PULocationID'] = df_february_filtered['PULocationID'].astype(str)
C:\Users\santa\AppData\Local\Temp\ipykernel_21832\3579443959.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_february_filtered['DOLocationID'] = df_february_filtered['DOLocationID'].astype(str)


In [16]:
data_dicts_val = df_february_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [17]:
feature_matrix_val = dv.transform(data_dicts_val)

In [18]:
y_val = df_february_filtered['duration']
y_val_pred = model.predict(feature_matrix_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
round(rmse_val, 2)

7.81